In [1]:
!pip install gensim

  Found existing installation: Cython 0.29.13
    Uninstalling Cython-0.29.13:
      Successfully uninstalled Cython-0.29.13


# Import

In [2]:
import re
import numpy as np 
import pandas as pd 

pd.set_option('display.max_colwidth', -1)

# dataset
from sklearn.datasets import fetch_20newsgroups

# Gensim packages
from gensim.parsing import strip_tags, strip_numeric, strip_multiple_whitespaces, stem_text, strip_punctuation, remove_stopwords
from gensim.parsing import preprocess_string

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """
C:\Users\User\Anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [3]:
# loading dataset
news_group = fetch_20newsgroups(subset='train')

news_group_data = news_group.data
news_group_target_names = news_group.target_names
news_group_target = news_group.target

In [4]:
# Creating a dataframe from the loaded data
news_df = pd.DataFrame({'news': news_group_data, 
                        'class': news_group_target})

In [9]:
news_df.head()

,news,class
0,"From: lerxst@wam.umd.edu (where's my thing)\nSubject: WHAT car is this!?\nNntp-Posting-Host: rac3.wam.umd.edu\nOrganization: University of Maryland, College Park\nLines: 15\n\n I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.\n\nThanks,\n- IL\n ---- brought to you by your neighborhood Lerxst ----\n\n\n\n\n",7
1,"From: guykuo@carson.u.washington.edu (Guy Kuo)\nSubject: SI Clock Poll - Final Call\nSummary: Final call for SI clock reports\nKeywords: SI,acceleration,clock,upgrade\nArticle-I.D.: shelley.1qvfo9INNc3s\nOrganization: University of Washington\nLines: 11\nNNTP-Posting-Host: carson.u.washington.edu\n\nA fair number of brave souls who upgraded their SI clock oscillator have\nshared their experiences for this poll. Please send a brief message detailing\nyour experiences with the procedure. Top speed attained, CPU rated speed,\nadd on cards and adapters, heat sinks, hour of usage per day, floppy disk\nfunctionality with 800 and 1.4 m floppies are especially requested.\n\nI will be summarizing in the next two days, so please add to the network\nknowledge base if you have done the clock upgrade and haven't answered this\npoll. Thanks.\n\nGuy Kuo <guykuo@u.washington.edu>\n",4
2,"From: twillis@ec.ecn.purdue.edu (Thomas E Willis)\nSubject: PB questions...\nOrganization: Purdue University Engineering Computer Network\nDistribution: usa\nLines: 36\n\nwell folks, my mac plus finally gave up the ghost this weekend after\nstarting life as a 512k way back in 1985. sooo, i'm in the market for a\nnew machine a bit sooner than i intended to be...\n\ni'm looking into picking up a powerbook 160 or maybe 180 and have a bunch\nof questions that (hopefully) somebody can answer:\n\n* does anybody know any dirt on when the next round of powerbook\nintroductions are expected? i'd heard the 185c was supposed to make an\nappearence ""this summer"" but haven't heard anymore on it - and since i\ndon't have access to macleak, i was wondering if anybody out there had\nmore info...\n\n* has anybody heard rumors about price drops to the powerbook line like the\nones the duo's just went through recently?\n\n* what's the impression of the display on the 180? i could probably swing\na 180 if i got the 80Mb disk rather than the 120, but i don't really have\na feel for how much ""better"" the display is (yea, it looks great in the\nstore, but is that all ""wow"" or is it really that good?). could i solicit\nsome opinions of people who use the 160 and 180 day-to-day on if its worth\ntaking the disk size and money hit to get the active display? (i realize\nthis is a real subjective question, but i've only played around with the\nmachines in a computer store breifly and figured the opinions of somebody\nwho actually uses the machine daily might prove helpful).\n\n* how well does hellcats perform? ;)\n\nthanks a bunch in advance for any info - if you could email, i'll post a\nsummary (news reading time is at a premium with finals just around the\ncorner... :( )\n--\nTom Willis \ twillis@ecn.purdue.edu \ Purdue Electrical Engineering\n---------------------------------------------------------------------------\n""Convictions are more dangerous enemies of truth than lies."" - F. W.\nNietzsche\n",4
3,"From: jgreen@amber (Joe Green)\nSubject: Re: Weitek P9000 ?\nOrganization: Harris Computer Systems Division\nLines: 14\nDistribution: world\nNNTP-Posting-Host: amber.ssd.csd.harris.com\nX-Newsreader: TIN [version 1.1 PL9]\n\nRobert J.C. Kyanko (rob@rjck.UUCP) wrote:\n> abraxis@iastate.edu writes in article <abraxis.734340159@cla

In [5]:
#randomly taking sample
news_extracts = news_df.sample(2000)

news_extracts.reset_index(drop=True, inplace=True)
news_extracts.head(2)

,news,class
0,"From: tedm@tsoft.net (Ted Matsumura)\nSubject: Re: Catalog of Hard-to-Find PC Enhancements (Repost)\nArticle-I.D.: tsoft.C5JCCG.3Bn\nDistribution: usa\nOrganization: TSoft BBS and Public Access Unix, +1 415 969 8238\nLines: 34\n\n[stuff deleted]\n>\n>For example, I subscribe to 2 magazines, ""ABC news"" and ""XYZ products"".\n>I give my name as ""Joseph X. Cool"" for the first, and ""Joe Q. Cool"" for\n>the 2nd. When I get junk mail addressed to ""Joseph X. Cool"", I know\n>where they bought my name.\n>\n>This doesn't help NOT getting junk mail, but you at least know who's\n>selling your name. And if you ask companies to NOT sell your name to\n>mailing lists/tele-marketeers, they are not supposed to (as in, it's\n>illegal.)\n>\n>And in case you haven't guessed, I don't really use Joe Cool.\n>\n>--\n\nMy company maintains a 20,000+ mailing list which is regularly rented for\none time use by the major software companies. The method you are using to\n""seed"" your junk mail, isn't really effective. Bulk mailers regulary \neither send their databases to be ""cleaned"" by the NCOA, which if you've\nmoved recently, will revert back to the original ""xxx Cool"", and in large\nmailings, there will likely be a dupe of you, and they'll pick the first, and use the\nothers for future mailings.\n \nBTW, our list is currently one of the hottest lists for actual buyers of\na MS Windows utility product in the $100 range, and is available through\nDirect Media in CT., at $0.10 per name. Please let your direct mail\nmarketing rep. know about this.. Thanks.\n \nTEd\n>\n> --- Matthew Caprile || Hey, I only speak for myself ! Even I ---\n> --- || won't admit to the opinions expressed ---\n> --- caprile@apple.com || here, so don't expect my employer to. ---\n",6
1,"From: hovig@uxa.cso.uiuc.edu (Hovig Heghinian)\nSubject: Re: Seventh Century A.D. Armenian Math Problems\nOrganization: University of Illinois at Urbana\nLines: 25\n\nkoc@rize.ECE.ORST.EDU (Cetin Kaya Koc) writes:\n\n>> Problem 1\n>> \n>> My father told me the following story. During the famous wars between the \n>> Armenians and the Persians, prince Zaurak Kamsarakan performed extraordinary\n>> heroic deeds. Three times in a single month he attacked the Persian troops. \n>> The first time, he struck down half of the Persian army. The second time, \n>> pursuing the Persians, he slaughtered one fourth of the soldiers. The third \n>> time, he destroyed one eleventh of the Persian army. The Persians who were \n>> still alive, numbering two hundred eighty, fled to Nakhichevan. And so, from\n>> this remainder, find how many Persian soldiers there were before the \n> massacre.\n>> \n\n>Answer: a(1-1/2-1/4-1/11)=280 -> a = 1760\n\n>Corollary: Armenians strike, slaughter, destroy, and massacre. After all,\n> they are not as innocent as the asala network claims.\n\n Hmm ... Turks sure know how to keep track of deaths, but they seem to\nlose count around 1.5 million.\n\nHovig\n\n",17


### Pre-Processing

In [6]:
# "\s"	Most engines: "whitespace character": space, tab, newline, carriage return,
# "\w" means "any word character" which usually means alphanumeric (letters, numbers, regardless of case) plus underscore (_)
# more info https://www.rexegg.com/regex-quickstart.html

In [7]:
# Custom filter method
transform_to_lower = lambda s: s.lower() #upper to lower case

remove_single_char = lambda s: re.sub(r'\s+\w{1}\s+', '', s) #regex to remove single char

# Filters to be executed in pipeline
CLEAN_FILTERS = [strip_tags,
                strip_numeric,
                strip_punctuation, 
                strip_multiple_whitespaces, 
                transform_to_lower,
                remove_stopwords,
                remove_single_char]

# Method does the filtering of all the unrelevant text elements
def cleaning_pipe(document):
    # Invoking gensim.parsing.preprocess_string method with set of filters
    processed_words = preprocess_string(document, CLEAN_FILTERS)
    
    return processed_words

In [8]:
# Apply the cleaning pipe on the news data

news_extracts['clean_text'] = news_extracts['news'].apply(cleaning_pipe)
news_extracts['clean_text'][0:2]

0    [tedm, tsoft, net, ted, matsumura, subject, catalog, hard, pc, enhancements, repost, articletsoft, cjccg, bn, distribution, usa, organization, tsoft, bbs, public, access, unix, lines, stuff, deleted, example, subscribe, magazines, abc, news, xyz, products, josephcool, joecool, nd, junk, mail, addressed, josephcool, know, boughthelp, getting, junk, mail, knowselling, ask, companies, sell, mailing, lists, tele, marketeers, supposedillegal, case, havenguesseduse, joe, cool, company, maintains, mailing, list, regularly, rented, time, use, major, software, companies, method, seed, junk, mail, isneffective, bulk, mailers, regulary, send, databases, cleaned, ncoa, ve, moved, recently, revert, original, xxx, cool, large, mailings, likely, dupe, ll, pick, use, future, mailings, btw, list, currently, hottest, ...]
1    [hovig, uxa, cso, uiuc, edu, hovig, heghinian, subject, seventh, centuryarmenian, math, problems, organization, university, illinois, urbana, lines, koc, rize, ece, orst, edu

In [10]:
#Stemming-Stemming is a technique of finding root word of the given word. 
#For example, if a word is 'running' then the stem word of that word is 'run'.

#Lemmatization-Lemmatization refers to find the axle word by doing vocabulary and morphological analysis of the words.